In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('battle_results.csv')

df['p1win'] = df['result'] == 1
df['p2win'] = df['result'] == 2
df['tie'] = df['result'] == 0

df['p1name'] = df['p1name'].str.title()
df['p2name'] = df['p2name'].str.title()

df = df.drop(columns=['Unnamed: 0'])

df.head(5)

,player1,player2,result,p1name,p2name,p1win,p2win,tie
0,0,0,2,Scyther,Scyther,False,True,False
1,0,1,1,Scyther,Pinsir,True,False,False
2,0,2,2,Scyther,Moltres,False,True,False
3,0,3,2,Scyther,Moltres Ex,False,True,False
4,0,4,2,Scyther,Heatmor,False,True,False


In [4]:
evo_names = df.groupby('player1')['p1name'].first().reset_index().rename(columns={'player1': 'id', 'p1name': 'name'})

evo_names.head(5)

,id,name
0,0,Scyther
1,1,Pinsir
2,2,Moltres
3,3,Moltres Ex
4,4,Heatmor


In [5]:
def get_losses(id:int) -> set[int]:
    p1 = df[(df['player1'] == id) & ~(df['p1win'])]
    p2 = df[(df['player2'] == id) & ~(df['p2win'])]
    return set([int(item) for item in set(p1['player2'].unique()) | set(p2['player1'].unique())])

In [6]:
loss74 = get_losses(74)
loss75 = get_losses(75)

eggLosses = loss74 & loss75

named_evo_list = lambda s: [evo_names.loc[item, 'name'].title() for item in s]

print(f"Common losses:           {named_evo_list(eggLosses)}")
print(f"Egg 74 unique losses:    {named_evo_list(loss74 - eggLosses)}")
print(f"Egg 75 unique losses:    {named_evo_list(loss75 - eggLosses)}")

Common losses:           ['Exeggutor Ex', 'Exeggutor Ex', 'Rampardos', 'Aerodactyl Ex', 'Infernape Ex']
Egg 74 unique losses:    ['Omastar', 'Rapidash', 'Gallade Ex']
Egg 75 unique losses:    ['Starmie Ex', 'Cresselia Ex', 'Exeggutor', 'Glaceon', 'Venusaur Ex', 'Mismagius Ex', 'Mismagius Ex', 'Rapidash', 'Rapidash', 'Celebi Ex', 'Dialga Ex']


In [7]:
max_battles = int((df['player1'].max() + 1) * 2) - 1
max_battles

615

In [8]:
(df['result'].value_counts() / df['result'].count() * 100).round(2)

result
2    55.27
1    44.57
0     0.16
Name: count, dtype: float64

In [9]:
player_wins = df.groupby('player1')[['p1win', 'tie']].sum().rename(columns={'p1win': 'player_win', 'tie': "player_tie"})
opponent_wins = df[df['player1'] != df['player2']].groupby('player2')[['p2win', 'tie']].sum().rename(columns={'p2win': 'opponent_win', 'tie': "opponent_tie"})

wins = player_wins.merge(opponent_wins, left_index=True, right_index=True).reset_index().rename(columns={'player1': 'id'})
wins['wins'] = wins['player_win'] + wins['opponent_win']

wins = wins.set_index('id').merge(evo_names.set_index('id'), right_index=True, left_index=True).reset_index()

wins['win_perc'] = (wins['wins'] / max_battles * 100).round(2)

wins.head(5)

,id,player_win,player_tie,opponent_win,opponent_tie,wins,name,win_perc
0,0,47,0,88,0,135,Scyther,21.95
1,1,137,0,202,0,339,Pinsir,55.12
2,2,152,0,236,0,388,Moltres,63.09
3,3,203,0,259,0,462,Moltres Ex,75.12
4,4,61,0,101,0,162,Heatmor,26.34


In [10]:
wins[['id', 'wins', 'win_perc', 'name']].sort_values('wins', axis=0, ascending=False).head(20)

,id,wins,win_perc,name
74,74,608,98.86,Exeggutor Ex
75,75,598,97.24,Exeggutor Ex
300,300,592,96.26,Rampardos
289,289,587,95.45,Magnezone
176,176,586,95.28,Aerodactyl Ex
286,286,586,95.28,Infernape Ex
290,290,584,94.96,Magnezone
99,99,570,92.68,Starmie Ex
72,72,566,92.03,Exeggutor
243,243,563,91.54,Blastoise Ex


In [11]:
list(set(named_evo_list(get_losses(195))))

['Machamp',
 'Mewtwo Ex',
 'Wormadam',
 'Dewgong',
 'Nidoking',
 'Electivire',
 'Yanmega Ex',
 'Glaceon',
 'Rapidash',
 'Vespiqueen',
 'Weavile Ex',
 'Garchomp',
 'Florges',
 'Exeggutor Ex',
 'Kabutops',
 'Cresselia',
 'Omastar',
 'Leafeon',
 'Torterra',
 'Mew Ex',
 'Primape',
 'Weezing',
 'Bellossom',
 'Infernape Ex',
 'Beedrill',
 'Pidgeot Ex',
 'Celebi Ex',
 'Darkrai Ex',
 'Serperior',
 'Magnezone',
 'Slurpuff',
 'Cresselia Ex',
 'Lumineon',
 'Scolipede',
 'Gengar',
 'Drednaw',
 'Gardevoir',
 'Gengar Ex',
 'Mamoswine',
 'Kangaskhan',
 'Victreebel',
 'Blastoise Ex',
 'Togekiss',
 'Regirock',
 'Dialga Ex',
 'Heliolisk',
 'Aerodactyl Ex',
 'Electrode',
 'Moltres Ex',
 'Exeggutor',
 'Marowak Ex',
 'Seaking',
 'Pincurchin',
 'Wigglytuff',
 'Drapion',
 'Eelektross',
 'Machamp Ex',
 'Gallade Ex',
 'Greninja',
 'Floatzel',
 'Bastidon',
 'Zapdos Ex',
 'Pidgeot',
 'Articuno Ex',
 'Dragonite',
 'Tangrowth',
 'Rampardos',
 'Gyarados Ex',
 'Starmie Ex']

In [12]:
df[(df['player1']==195) & (df['p1win']) & (df['p2name'].str.contains(" ex", case=False)) | ((df['player2']==195) & (df['p2win'])  & (df['p1name'].str.contains(" ex", case=False)))]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
1119,3,195,2,Moltres Ex,Floatzel,False,True,False
2351,7,195,2,Articuno Ex,Floatzel,False,True,False
3275,10,195,2,Pikachu Ex,Floatzel,False,True,False
9127,29,195,2,Celebi Ex,Floatzel,False,True,False
10359,33,195,2,Mew Ex,Floatzel,False,True,False
14671,47,195,2,Palkia Ex,Floatzel,False,True,False
15287,49,195,2,Pachirisu Ex,Floatzel,False,True,False
21139,68,195,2,Lapras Ex,Floatzel,False,True,False
21447,69,195,2,Cresselia Ex,Floatzel,False,True,False
25143,81,195,2,Arcanine Ex,Floatzel,False,True,False


In [13]:
df[((df['player1'] == 75) & ~(df['p1win'])) | ((df['player2'] == 75) & ~(df['p2win']))]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
9007,29,75,1,Celebi Ex,Exeggutor Ex,True,False,False
22251,72,75,1,Exeggutor,Exeggutor Ex,True,False,False
23163,75,63,2,Exeggutor Ex,Dialga Ex,False,True,False
23169,75,69,2,Exeggutor Ex,Cresselia Ex,False,True,False
23174,75,74,2,Exeggutor Ex,Exeggutor Ex,False,True,False
23175,75,75,2,Exeggutor Ex,Exeggutor Ex,False,True,False
23183,75,83,2,Exeggutor Ex,Rapidash,False,True,False
23199,75,99,2,Exeggutor Ex,Starmie Ex,False,True,False
23332,75,232,2,Exeggutor Ex,Venusaur Ex,False,True,False
30567,99,75,1,Starmie Ex,Exeggutor Ex,True,False,False


In [14]:
wins[['id', 'player_win', 'name']].sort_values('player_win', axis=0, ascending=False).head(20)

,id,player_win,name
300,300,307,Rampardos
74,74,307,Exeggutor Ex
176,176,303,Aerodactyl Ex
75,75,301,Exeggutor Ex
290,290,300,Magnezone
289,289,298,Magnezone
286,286,292,Infernape Ex
99,99,286,Starmie Ex
234,234,284,Beedrill
72,72,282,Exeggutor


In [15]:
df[df['tie']].groupby('p1name')['result'].count().reset_index().sort_values('result', ascending=False)

,p1name,result
12,Lumineon,24
6,Fan Rotom,15
9,Hitmonlee,13
0,Aerodactyl,13
23,Zebstrika,13
1,Chatot,11
15,Mespirit,11
14,Manaphy,11
4,Ditto,10
16,Mew Ex,9


In [16]:
wins[wins['wins'] <= 5][['id', 'name', 'wins']].sort_values('wins', ascending=False)

,id,name,wins
165,165,Lumineon,4
19,19,Hitmonlee,1
39,39,Chatot,0
48,48,Manaphy,0
52,52,Mespirit,0
66,66,Fan Rotom,0
147,147,Aerodactyl,0
166,166,Lumineon,0
